In [2]:
import torch

# 檢查系統中是否有可用的 CUDA
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available. Using CPU instead.")

CUDA is available!


In [3]:
# %load train.py
from argparse import ArgumentParser
from typing import Tuple


import numpy as np
import torch
import torch.nn.functional as F
from pytorch_lightning import seed_everything, Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.types import STEP_OUTPUT, EPOCH_OUTPUT

from dataset.mpii_face_gaze_dataset import get_dataloaders
from model import FinalModel
from utils import calc_angle_error, PitchYaw, plot_prediction_vs_ground_truth, log_figure, get_random_idx, get_each_of_one_grid_idx




INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


In [4]:
class Model(FinalModel):
    def __init__(self, learning_rate: float = 0.001, weight_decay: float = 0., k=None, adjust_slope: bool = False, grid_calibration_samples: bool = False, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.k = [9, 128] if k is None else k
        self.adjust_slope = adjust_slope
        self.grid_calibration_samples = grid_calibration_samples

        self.save_hyperparameters()  # log hyperparameters

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)

    def __step(self, batch: dict) -> Tuple:
        """
        Operates on a single batch of data.

        :param batch: The output of your :class:`~torch.utils.data.DataLoader`. A tensor, tuple or list.
        :return: calculated loss, given values and predicted outputs
        """
        person_idx = batch['person_idx'].long()
        left_eye_image = batch['left_eye_image'].float()
        right_eye_image = batch['right_eye_image'].float()
        full_face_image = batch['full_face_image'].float()

        gaze_pitch = batch['gaze_pitch'].float()
        gaze_yaw = batch['gaze_yaw'].float()
        labels = torch.stack([gaze_pitch, gaze_yaw]).T

        outputs = self(person_idx, full_face_image, right_eye_image, left_eye_image)  # prediction on the base model
        loss = F.mse_loss(outputs, labels)

        return loss, labels, outputs

    def training_step(self, train_batch: dict, batch_idx: int) -> STEP_OUTPUT:
        loss, labels, outputs = self.__step(train_batch)

        self.log('train/loss', loss)
        self.log('train/angular_error', calc_angle_error(labels, outputs))

        return loss

    def validation_step(self, valid_batch: dict, batch_idx: int) -> STEP_OUTPUT:
        loss, labels, outputs = self.__step(valid_batch)

        self.log('valid/offset(k=0)/loss', loss)
        self.log('valid/offset(k=0)/angular_error', calc_angle_error(labels, outputs))

        return {'loss': loss, 'labels': labels, 'outputs': outputs, 'gaze_locations': valid_batch['gaze_location'], 'screen_sizes': valid_batch['screen_size']}

    def validation_epoch_end(self, outputs: EPOCH_OUTPUT) -> None:
        self.__log_and_plot_details(outputs, 'valid')

    def test_step(self, test_batch: dict, batch_idx: int) -> STEP_OUTPUT:
        loss, labels, outputs = self.__step(test_batch)

        self.log('test/offset(k=0)/loss', loss)
        self.log('test/offset(k=0)/angular_error', calc_angle_error(labels, outputs))

        return {'loss': loss, 'labels': labels, 'outputs': outputs, 'gaze_locations': test_batch['gaze_location'], 'screen_sizes': test_batch['screen_size']}

    def test_epoch_end(self, outputs: EPOCH_OUTPUT) -> None:
        self.__log_and_plot_details(outputs, 'test')

    def __log_and_plot_details(self, outputs, tag: str):
        test_labels = torch.cat([output['labels'] for output in outputs])
        test_outputs = torch.cat([output['outputs'] for output in outputs])
        test_gaze_locations = torch.cat([output['gaze_locations'] for output in outputs])
        test_screen_sizes = torch.cat([output['screen_sizes'] for output in outputs])

        figure = plot_prediction_vs_ground_truth(test_labels, test_outputs, PitchYaw.PITCH)
        log_figure(self.logger, f'{tag}/offset(k=0)/pitch', figure, self.global_step)

        figure = plot_prediction_vs_ground_truth(test_labels, test_outputs, PitchYaw.YAW)
        log_figure(self.logger, f'{tag}/offset(k=0)/yaw', figure, self.global_step)

        # find calibration params
        last_x = 500
        calibration_train = test_outputs[:-last_x].cpu().detach().numpy()
        calibration_test = test_outputs[-last_x:].cpu().detach().numpy()

        calibration_train_labels = test_labels[:-last_x].cpu().detach().numpy()
        calibration_test_labels = test_labels[-last_x:].cpu().detach().numpy()

        gaze_locations_train = test_gaze_locations[:-last_x].cpu().detach().numpy()
        screen_sizes_train = test_screen_sizes[:-last_x].cpu().detach().numpy()

        if len(calibration_train) > 0:
            for k in self.k:
                if k <= 0:
                    continue
                calibrated_solutions = []

                num_calibration_runs = 500 if self.grid_calibration_samples else 10_000  # original results are both evaluated with 10,000 runs
                for calibration_run_idx in range(num_calibration_runs):  # get_each_of_one_grid_idx is slower than get_random_idx
                    np.random.seed(42 + calibration_run_idx)
                    calibration_sample_idxs = get_each_of_one_grid_idx(k, gaze_locations_train, screen_sizes_train) if self.grid_calibration_samples else get_random_idx(k, len(calibration_train))
                    calibration_points_x = np.asarray([calibration_train[idx] for idx in calibration_sample_idxs])
                    calibration_points_y = np.asarray([calibration_train_labels[idx] for idx in calibration_sample_idxs])

                    if self.adjust_slope:
                        m, b = np.polyfit(calibration_points_y[:, :1].reshape(-1), calibration_points_x[:, :1].reshape(-1), deg=1)
                        pitch_fixed = (calibration_test[:, :1] - b) * (1 / m)
                        m, b = np.polyfit(calibration_points_y[:, 1:].reshape(-1), calibration_points_x[:, 1:].reshape(-1), deg=1)
                        yaw_fixed = (calibration_test[:, 1:] - b) * (1 / m)
                    else:
                        mean_diff_pitch = (calibration_points_y[:, :1] - calibration_points_x[:, :1]).mean()  # mean offset
                        pitch_fixed = calibration_test[:, :1] + mean_diff_pitch
                        mean_diff_yaw = (calibration_points_y[:, 1:] - calibration_points_x[:, 1:]).mean()  # mean offset
                        yaw_fixed = calibration_test[:, 1:] + mean_diff_yaw

                    pitch_fixed, yaw_fixed = torch.Tensor(pitch_fixed), torch.Tensor(yaw_fixed)
                    outputs_fixed = torch.stack([pitch_fixed, yaw_fixed], dim=1).squeeze(-1)
                    calibrated_solutions.append(calc_angle_error(torch.Tensor(calibration_test_labels), outputs_fixed).item())

                self.log(f'{tag}/offset(k={k})/mean_angular_error', np.asarray(calibrated_solutions).mean())
                self.log(f'{tag}/offset(k={k})/std_angular_error', np.asarray(calibrated_solutions).std())

        # best case, with all calibration samples, all values except the last `last_x` values
        if self.adjust_slope:
            m, b = np.polyfit(calibration_train_labels[:, :1].reshape(-1), calibration_train[:, :1].reshape(-1), deg=1)
            pitch_fixed = torch.Tensor((calibration_test[:, :1] - b) * (1 / m))
            m, b = np.polyfit(calibration_train_labels[:, 1:].reshape(-1), calibration_train[:, 1:].reshape(-1), deg=1)
            yaw_fixed = torch.Tensor((calibration_test[:, 1:] - b) * (1 / m))
        else:
            mean_diff_pitch = (calibration_train_labels[:, :1] - calibration_train[:, :1]).mean()  # mean offset
            pitch_fixed = calibration_test[:, :1] + mean_diff_pitch
            mean_diff_yaw = (calibration_train_labels[:, 1:] - calibration_train[:, 1:]).mean()  # mean offset
            yaw_fixed = calibration_test[:, 1:] + mean_diff_yaw

        pitch_fixed, yaw_fixed = torch.Tensor(pitch_fixed), torch.Tensor(yaw_fixed)
        outputs_fixed = torch.stack([pitch_fixed, yaw_fixed], dim=1).squeeze(-1)
        calibration_test_labels = torch.Tensor(calibration_test_labels)
        self.log(f'{tag}/offset(k=all)/angular_error', calc_angle_error(calibration_test_labels, outputs_fixed))

        figure = plot_prediction_vs_ground_truth(calibration_test_labels, outputs_fixed, PitchYaw.PITCH)
        log_figure(self.logger, f'{tag}/offset(k=all)/pitch', figure, self.global_step)

        figure = plot_prediction_vs_ground_truth(calibration_test_labels, outputs_fixed, PitchYaw.YAW)
        log_figure(self.logger, f'{tag}/offset(k=all)/yaw', figure, self.global_step)

    




In [5]:
def main(path_to_data: str, validate_on_person: int, test_on_person: int, learning_rate: float, weight_decay: float, batch_size: int, k: int, adjust_slope: bool, grid_calibration_samples: bool):
    seed_everything(42)

    model = Model(learning_rate, weight_decay, k, adjust_slope, grid_calibration_samples)

    trainer = Trainer(
        gpus=1,
        max_epochs=50,
        default_root_dir='./saved_models/',
        logger=[
            TensorBoardLogger(save_dir="tb_logs"),
        ],
        benchmark=True,
    )

    train_dataloader, valid_dataloader, test_dataloader = get_dataloaders(path_to_data, validate_on_person, test_on_person, batch_size)
    trainer.fit(model, train_dataloader, valid_dataloader)
    trainer.test(model, test_dataloader)




In [6]:
import argparse
import sys

path_to_data = './data/mpiifacegaze_preprocessed'
validate_on_person = 1
test_on_person = 0
learning_rate = 0.001
weight_decay = 0
batch_size = 64
k = [9, 128]
adjust_slope = False
grid_calibration_samples = False

main(path_to_data, validate_on_person, test_on_person, learning_rate, weight_decay, batch_size, k, adjust_slope, grid_calibration_samples)

Global seed set to 42
C:\Users\User\anaconda3\envs\gazegpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\User\anaconda3\envs\gazegpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


train on persons [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
valid on person 1
test on person 0
len(dataset_train) 60784
len(dataset_train) 2904


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


len(dataset_train) 2927



  | Name         | Type       | Params
--------------------------------------------
0 | cnn_face     | Sequential | 564 K 
1 | cnn_eye      | Sequential | 564 K 
2 | fc_face      | Sequential | 1.2 M 
3 | cnn_eye2fc   | Sequential | 904 K 
4 | fc_eye       | Sequential | 1.6 M 
5 | fc_eyes_face | Sequential | 148 K 
--------------------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.0 M     Total params
19.815    Total estimated model params size (MB)


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:09<00:00,  3.98s/it]

C:\Users\User\AppData\Local\Temp\ipykernel_9228\3126367832.py:127: RuntimeWarning: Mean of empty slice.
  mean_diff_pitch = (calibration_train_labels[:, :1] - calibration_train[:, :1]).mean()  # mean offset
C:\Users\User\anaconda3\envs\gazegpu\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\AppData\Local\Temp\ipykernel_9228\3126367832.py:129: RuntimeWarning: Mean of empty slice.
  mean_diff_yaw = (calibration_train_labels[:, 1:] - calibration_train[:, 1:]).mean()  # mean offset


Epoch 1:  95%|█████████▌| 950/996 [07:31<00:21,  2.11it/s, loss=0.00664, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 2:  95%|█████████▌| 950/996 [06:50<00:19,  2.32it/s, loss=0.00533, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 3:  95%|█████████▌| 950/996 [06:35<00:19,  2.40it/s, loss=0.00549, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 4:  95%|█████████▌| 950/996 [06:29<00:18,  2.44it/s, loss=0.00421, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 5:  95%|█████████▌| 950/996 [06:36<00:19,  2.40it/s, loss=0.00415, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 6:  95%|█████████▌| 950/996 [06:30<00:18,  2.44it/s, loss=0.00352, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 7:  95%|█████████▌| 950/996 [06:36<00:19,  2.40it/s, loss=0.00329, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 8:  95%|█████████▌| 950/996 [06:35<00:19,  2.40it/s, loss=0.00289, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 9:  95%|█████████▌| 950/996 [06:33<00:19,  2.41it/s, loss=0.00298, v_num=2]
Validation: 0i

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 46/46 [00:16<00:00,  2.77it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             Test metric                         DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test/offset(k=0)/angular_error             3.5704545974731445
        test/offset(k=0)/loss                0.0024097885470837355
test/offset(k=128)/mean_angular_error         1.9371178730130196
test/offset(k=128)/std_angular_error          0.03925926930567316
 test/offset(k=9)/mean_angular_error           2.044651872718334
 test/offset(k=9)/std_angular_error           0.21102711369752403
  test/offset(k=all)/angular_error            1.9292210340499878
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
